<a href="https://colab.research.google.com/github/ReddyGopichand/Credit-Card-Validator/blob/main/Copy_of_today.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
from bs4 import BeautifulSoup

def avg_f(l):
    if len(l) != 5:
        raise Exception("not 5")
    base = l[-2]
    recent_quarters = l[:3]
    if base == 0:
        return
    per_inc = [(q-base)/base for q in recent_quarters]
    return sum(per_inc)/len(per_inc)

def get_net_profit(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    tags = soup.select('#C-3-income-graphData')
    tag = tags[0]
    data = json.loads(tag.contents[0])
    return data['net-profit']

def get_company_scid(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.select('#ap_sc_id')
    try:
        return tags[0].get_attribute_list('value')
    except:
        return 'defaultscid'

def get_companies(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    selector = '.scroll-pane.MT5'
    tags = soup.select(selector)
    todays, tomorrows = tags[0], tags[1]
    companies_list = []
    for anchor in tomorrows.select('table tr td:first-child a'):
        code = get_company_scid(anchor.get_attribute_list('href')[0])
        if type(code) == list:
            code = code[0]
        company_name = anchor.contents[0]
        companies_list.append((code,company_name))
    return companies_list

def get_url_content(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f'response code {response.status_code}')
    except Exception as error_in_get_url_content:
        print("error_in_get_url_content",error_in_get_url_content)

def main():
    companies_url = 'https://www.moneycontrol.com/markets/earnings/?classic=true'
    output_result = []
    companies_list = get_companies(get_url_content(companies_url))
    print(len(companies_list),'company results')
    missed_companies_list = []
    request_types="CS"
    for company_code, company_name in companies_list:
        for request_type in request_types:
            try:
                url = 'https://www.moneycontrol.com/mc/widget/mcfinancials/getFinancialData?classic=true&referenceId=income&requestType='+request_type+'&scId='+company_code+'&frequency=3'
                net_profit = [int(i) for i in get_net_profit(get_url_content(url))]
                avg_inc = avg_f(net_profit)
                if avg_inc:
                    output_result.append((int(avg_inc*100),company_name))
                break
            except Exception as e:
                continue
        if len(output_result)==0 or output_result[-1][1]!=company_name:
            missed_companies_list.append(company_name)
    output_result.sort(reverse=True)
    print("missed",missed_companies_list)
    print(output_result,len(output_result))
main()

664 company results
missed ['Aadhaar Venture', 'Aagam Capital', 'Aananda Lakshmi', 'Aarey Drugs', 'Abhinav Capital', 'Abhinav Leasing', 'Abhishek Corp', 'Abhishek Infra', 'ACE ENG', 'Adarsh Mercant', 'Add-Shop Promot', 'ADINATH TEXTILE', 'Adishakti Loha', 'Aditya Ispat', 'ADS Diagnostic ', 'Advik Labors', 'Ajwa Fun World', 'Alka India', 'Allied Computer', 'Alpine Hsg', 'Ambitious Plast', 'Amforge', 'Amsons Apparels', 'Anand Rayons', 'Ansal Buildwell', 'Apex Capital An', 'APL', 'Aptus Ind', 'ARC Finance', 'Ardi Investment', 'Arunjyoti Bio', 'Aryaman Fin Ser', 'AS&SBL', 'Ascensive', 'Ashika Credit', 'Ashutosh Paper ', 'Asia Capital', 'Asit C Mehta', 'ATN Int', 'Ausom Enterp', 'Automobile Prod', 'AVAILABLE FINAN', 'Avasara Finance', 'Axel Polymers', 'B P Capital', 'B Right Realest', 'Bambino Agro', 'Bangalore Fort', 'Bella Casa', 'Bentley Commerc', 'Bervin Invest', 'Beryl Drugs', 'Betex', 'Bhagyashree Lea', 'Bhakti Gems', 'Bharat Texts', 'Bhaskar Agroche', 'Bhatia Communic', 'Binny Mills'